##### This notebook demonstrates a demo of how you can deploy your first job with Truefoundry.

---


After you complete the notebook, you will have a successful deployed a job to train a model on the iris dataser. Your jobs deployment dashboard will look like this:

![](https://files.readme.io/2f6871c-Screenshot_2022-11-16_at_11.43.31_PM.png)


## Project structure

To complete this guide, you are going to create the following **files**:

- `train.py` : contains our training code
- `requirements.txt` : contains our dependencies
- `deploy.py` contains our deployment code ( you can also use a deployment configuration for deploying using a YAML file)

Your **final file structure** is going to look like this:

```
.
├── train.py
├── deploy.py
└── requirements.txt
```

As you can see, all the following files are created in the same folder/directory

**Let's create the directory which will contain all this files:-**


In [ ]:
%mkdir develop
%cd develop

/content/develop


# Setup

Let's first setup all the things we need to deploy our service.

- Signup or Login on TrueFoundry
- Setup ServiceFoundry
- Setup Workspace


Let's start with installing servicefoundry.


In [ ]:
!pip install -U "servicefoundry"

In [ ]:
import logging

[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(name)s] %(levelname)-8s %(message)s"
)

In [ ]:
!sfy login --relogin

### Workspace FQN

Once you run the cell below you will get a prompt to enter your workspace. <br>

- Step 1: Click on the link given in the prompt.
- Step 2: Identify the Workspace you want to deploy the application in.
- Step 3: Copy the Workspace FQN <br>
  ![Copying Workspace FQN](https://files.readme.io/730fee2-Screenshot_2023-02-28_at_2.08.34_PM.png)
- Step 4: Paste the Workspace FQN in the prompt and press enter.


In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

WORKSPACE_FQN = getpass(
    "Please enter you WORKSPACE_FQN (https://app.truefoundry.com/workspaces):"
)

### API KEY

Once you run the cell below you will get a prompt to enter your API KEY. <br>

- Step 1: Click on the link given in the prompt.
- Step 2: Click on Create New API Key button
- Step 3: Enter the name you want to give to the API KEY and then click on generate
- Step 4: Copy the API KEY, You can also download the API KEY for later use <br>
  ![Copying API KEY](https://files.readme.io/201c8aa-Screenshot_2023-02-28_at_2.28.17_PM.png)
- Step 5: Paste the API KEY in the prompt and press enter.


In [ ]:
TFY_API_KEY = getpass(
    "Please enter your API Key (https://app.truefoundry.com/settings):"
)

## Step 1: Implement the training code

The first step is to create a job that trains a scikit learn model on iris dataset

We start with a `train.py` containing our training code and `requirements.txt` with our dependencies.

```
.
├── train.py
└── requirements.txt
```


### **`train.py`**


In [ ]:
%%writefile train.py
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# load the dataset
X, y = load_iris(as_frame=True, return_X_y=True)
X = X.rename(columns={
        "sepal length (cm)": "sepal_length",
        "sepal width (cm)": "sepal_width",
        "petal length (cm)": "petal_length",
        "petal width (cm)": "petal_width",
})

# NOTE:- You can pass these configurations via command line
# arguments, config file, environment variables.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Initialize the model
clf = LogisticRegression(solver="liblinear")
# Fit the model
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
print(classification_report(y_true=y_test, y_pred=preds))

Click on this [link](https://docs.truefoundry.com/v0.1.1/recipes/training-a-scikit-learn-model) to understand the **`app.py`**:


### **`requirements.txt`**


In [ ]:
%%writefile requirements.txt
pandas==1.4.4
numpy==1.22.4
scikit-learn==1.2.2

# for deploying our job deployments
servicefoundry==0.10.1

## Step 2: Deploying as a Job

You can deploy services on Truefoundry programmatically via our **Python SDK**.

Create the `deploy.py`, after which our file structure will look like this:

**File Structure**

```Text Text
.
├── train.py
├── deploy.py
└── requirements.txt
```

### **`deploy.py`**


In [ ]:
%%writefile deploy.py
import argparse
import logging
from servicefoundry import Build, Job, PythonBuild

logging.basicConfig(level=logging.INFO)

parser = argparse.ArgumentParser()
parser.add_argument("--workspace_fqn", required=True, type=str)
args = parser.parse_args()

# First we define how to build our code into a Docker image
image = Build(
    build_spec=PythonBuild(
        command="python train.py",
        requirements_path="requirements.txt",
    )
)

job = Job(
    name="iris-train-job",
    image=image
)
job.deploy(workspace_fqn=args.workspace)

Click on this [link](https://docs.truefoundry.com/recipes/deploy-training-code-as-a-job) to understand the **`deploy.py`**:


Now to deploy our Job run the command below


In [ ]:
!python deploy.py --workspace_fqn $WORKSPACE_FQN